building the iterator is almost as hard as building the model..

In [104]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import glob

import torch
import torch.nn as nn
import torch.optim as optim

from torchtext import data
from torchtext import datasets

import spacy
import numpy as np

import time
import random


from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [118]:
TAGS = data.Field(unk_token = None)
TAGS.build_vocab(['0','S', 'C','G','T','B','D', 'A'])

In [119]:
TAGS.numericalize(['0'])

tensor([[1]])

In [229]:
path= '/Users/tancredicp/Desktop/sailing_app/data/ML Training Resources/'
all_files = glob.glob(path + "/*.csv")


# listOfNormalizedDatFrames = []
# listofTags = []

Data = []

counter = -1

for filename in all_files:
    temporary_dict = {}

    
    
    dfSpecific = pd.read_csv(filename, index_col=None, header=0)
#     print(dfSpecific.shape)
    #I was lazy and did not check where the NaNs where but they were being annoying so made them all 0s.
    dfSpecific.fillna(0, inplace= True)
    
    dfSpecificY = dfSpecific[['tag']]
    dfSpecificY = dfSpecificY['tag'] #.astype('category')
    
    
    temporary_dict['tags'] = TAGS.numericalize((np.array(dfSpecificY)))
    

    
    dfSpecificX = dfSpecific.drop(columns=['tag', 'tag_name', 'boat_class', 'time'])
    scaler = StandardScaler()
    columnsBeingStandardize = ['lat', 'lon', 'sog', 'heel', 'clew_load','pitch']
    dfSpecificX[columnsBeingStandardize] = scaler.fit_transform(dfSpecificX[columnsBeingStandardize])
    
    
    array1 = np.array(dfSpecificX[['lat', 'lon', 'sog', 'cog']])
    
    print(torch.from_numpy(array1).type(torch.DoubleTensor))
    
    temporary_dict['track'] = torch.from_numpy(array1).type(torch.DoubleTensor)
#     print(temporary_dict)
    
    Data.append(temporary_dict)
    
#     listOfNormalizedDatFrames[counter] = array1
    
#     break

# X = pd.concat(listOfNormalizedDatFrames, axis=0, ignore_index=True)
# y = pd.concat(listofTags, axis=0, ignore_index=True)

tensor([[ 0.5169, -0.5162,  1.3716, -0.3000],
        [ 0.5262, -0.5372,  1.5119, -0.3111],
        [ 0.5362, -0.5570,  1.4558, -0.2722],
        ...,
        [ 0.9956, -1.3453, -1.1912, -0.3667],
        [ 0.9956, -1.3475, -1.1459, -0.4167],
        [ 0.9958, -1.3498, -1.1697, -0.4389]], dtype=torch.float64)
tensor([[ 1.0801, -0.8038, -1.6933,  0.3556],
        [ 1.1136, -0.7263, -1.6693,  0.2056],
        [ 1.0522, -0.3454,  0.1338,  0.6667],
        ...,
        [ 0.8140, -1.7687, -1.7196, -0.4778],
        [ 0.8146, -1.7702, -1.5906, -0.3167],
        [ 0.8147, -1.7718, -1.6453, -0.4111]], dtype=torch.float64)
tensor([[ 0.9253, -0.1155, -1.2771,  0.3389],
        [ 0.9258, -0.1135, -1.2578,  0.3667],
        [ 0.9263, -0.1116, -1.2578,  0.3333],
        ...,
        [ 0.9826, -1.1953,  0.4812, -0.7722],
        [ 0.9764, -1.2083,  0.5179, -0.7556],
        [ 0.9702, -1.2210,  0.4602, -0.7611]], dtype=torch.float64)
tensor([[ 1.3487,  0.7451,  0.1557,  0.8389],
        [ 1.3186,  0.

In [208]:
# Data

In [209]:
from torch.utils.data import Dataset

In [210]:
class SailingData(Dataset):
    def __init__ (self, pl):
        self.samples = []
        
        for d in pl:
            self.samples.append((d['track'], d['tags']))
            
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
            return(self.samples[idx])

In [211]:
BATCH_SIZE = 1
Dataset1= SailingData(Data)

In [212]:
Dataset1[0]

(tensor([[ 0.5169, -0.5162,  1.3716, -0.3000],
         [ 0.5262, -0.5372,  1.5119, -0.3111],
         [ 0.5362, -0.5570,  1.4558, -0.2722],
         ...,
         [ 0.9956, -1.3453, -1.1912, -0.3667],
         [ 0.9956, -1.3475, -1.1459, -0.4167],
         [ 0.9958, -1.3498, -1.1697, -0.4389]], dtype=torch.float64),
 tensor([[7, 7, 7,  ..., 4, 4, 4]]))

In [213]:
train_iterator = torch.utils.data.DataLoader(Dataset1)

In [214]:
train_iterator

In [215]:
class BiLSTMPOSTagger(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hidden_dim, 
                 output_dim, 
                 n_layers, 
                 bidirectional, 
                 dropout):
        
        super().__init__()
        
        
        self.lstm = nn.LSTM(input_dim, 
                            hidden_dim, 
                            num_layers = n_layers, 
                            bidirectional = bidirectional)
                           # dropout = dropout if n_layers > 0.9 else 0.1)
        
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):

       
        print("inside", text)
        
        outputs, (hidden, cell) = self.lstm(text)
 
        predictions = self.fc(self.dropout(outputs))
        
        
        return predictions

In [216]:
INPUT_DIM = 4
HIDDEN_DIM = 32
OUTPUT_DIM = len(TAGS.vocab)
N_LAYERS = 2
BIDIRECTIONAL = False
DROPOUT = 0.5

model = BiLSTMPOSTagger(INPUT_DIM,  
                        HIDDEN_DIM, 
                        OUTPUT_DIM, 
                        N_LAYERS, 
                        BIDIRECTIONAL, 
                        DROPOUT)

In [217]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean = 0, std = 0.1)
        
model.apply(init_weights)

BiLSTMPOSTagger(
  (lstm): LSTM(4, 32, num_layers=2)
  (fc): Linear(in_features=32, out_features=9, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [218]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 13,609 trainable parameters


In [219]:
optimizer = optim.Adam(model.parameters())


In [220]:
def categorical_accuracy(preds, y, tag_pad_idx):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]])

In [221]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    print(iterator)
    for batch in iterator:
        
        text = batch[0]
        tags = batch[1]
        
        
        print('text', text)
        print('tag', tags)
        
        
        optimizer.zero_grad()
        
        
        predictions = model(text)
        
 
        
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)
        

        loss = criterion(predictions, tags)
                
        acc = categorical_accuracy(predictions, tags)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [222]:
criterion = nn.CrossEntropyLoss()

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [223]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')

text tensor([[[ 0.5169, -0.5162,  1.3716, -0.3000],
         [ 0.5262, -0.5372,  1.5119, -0.3111],
         [ 0.5362, -0.5570,  1.4558, -0.2722],
         ...,
         [ 0.9956, -1.3453, -1.1912, -0.3667],
         [ 0.9956, -1.3475, -1.1459, -0.4167],
         [ 0.9958, -1.3498, -1.1697, -0.4389]]], dtype=torch.float64)
tag tensor([[[7, 7, 7,  ..., 4, 4, 4]]])
inside tensor([[[ 0.5169, -0.5162,  1.3716, -0.3000],
         [ 0.5262, -0.5372,  1.5119, -0.3111],
         [ 0.5362, -0.5570,  1.4558, -0.2722],
         ...,
         [ 0.9956, -1.3453, -1.1912, -0.3667],
         [ 0.9956, -1.3475, -1.1459, -0.4167],
         [ 0.9958, -1.3498, -1.1697, -0.4389]]], dtype=torch.float64)


RuntimeError: Expected object of scalar type Double but got scalar type Float for argument #3 'mat2' in call to _th_addmm_out

In [206]:
for x in train_iterator:
    print(x)

[tensor([[[ 0.5169, -0.5162,  1.3716, -0.3000],
         [ 0.5262, -0.5372,  1.5119, -0.3111],
         [ 0.5362, -0.5570,  1.4558, -0.2722],
         ...,
         [ 0.9956, -1.3453, -1.1912, -0.3667],
         [ 0.9956, -1.3475, -1.1459, -0.4167],
         [ 0.9958, -1.3498, -1.1697, -0.4389]]], dtype=torch.float64), tensor([[[7, 7, 7,  ..., 4, 4, 4]]])]
[tensor([[[ 1.0801, -0.8038, -1.6933,  0.3556],
         [ 1.1136, -0.7263, -1.6693,  0.2056],
         [ 1.0522, -0.3454,  0.1338,  0.6667],
         ...,
         [ 0.8140, -1.7687, -1.7196, -0.4778],
         [ 0.8146, -1.7702, -1.5906, -0.3167],
         [ 0.8147, -1.7718, -1.6453, -0.4111]]], dtype=torch.float64), tensor([[[2, 2, 2,  ..., 4, 4, 4]]])]
[tensor([[[ 0.9253, -0.1155, -1.2771,  0.3389],
         [ 0.9258, -0.1135, -1.2578,  0.3667],
         [ 0.9263, -0.1116, -1.2578,  0.3333],
         ...,
         [ 0.9826, -1.1953,  0.4812, -0.7722],
         [ 0.9764, -1.2083,  0.5179, -0.7556],
         [ 0.9702, -1.2210,  0.4